<a href="https://colab.research.google.com/github/gopal2812/convandgpt/blob/main/session7/s7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/gopal2812/convandgpt.git
import os

Cloning into 'convandgpt'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 5), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (23/23), 122.88 KiB | 13.65 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
os.chdir("/content/convandgpt/session7")

In [3]:
!pwd;ls

/content/convandgpt/session7
models.py  utils.py


In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from models import Model_1, Model_2, Model_3
import utils

In [5]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [6]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 191995711.03it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 42955919.80it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 108233299.89it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5370884.91it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
torch.manual_seed(1)
use_cuda = torch.cuda.is_available()
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 198659750.50it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 37215266.92it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 68508765.78it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4809525.06it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



## Model 1

In [9]:
# !pip install torchsummary
from torchsummary import summary
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_1().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             160
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 10, 10, 10]             900
             ReLU-12           [-1, 10, 10, 10]               0
      BatchNorm2d-13           [-1, 10, 10, 10]              20
          Dropout-14           [-1, 10,

In [10]:
model = Model_1().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(0, 15):
    print(f"EPOCH: {epoch+1}")
    utils.train(model, device, train_loader, optimizer, epoch)
    utils.test(model, device, test_loader)

EPOCH: 1


Loss=0.302359402179718 Batch_id=937 Accuracy=81.28: 100%|██████████| 938/938 [00:21<00:00, 43.10it/s]



Test set: Average loss: 0.1856, Accuracy: 9541/10000 (95.41%)

EPOCH: 2


Loss=0.36516356468200684 Batch_id=937 Accuracy=86.12: 100%|██████████| 938/938 [00:24<00:00, 38.22it/s]



Test set: Average loss: 0.1181, Accuracy: 9721/10000 (97.21%)

EPOCH: 3


...


Test set: Average loss: 0.0598, Accuracy: 9856/10000 (98.56%)

EPOCH: 14


Loss=0.6060293316841125 Batch_id=937 Accuracy=88.45: 100%|██████████| 938/938 [00:20<00:00, 44.87it/s]



Test set: Average loss: 0.0501, Accuracy: 9889/10000 (98.89%)

EPOCH: 15


Loss=0.7331061959266663 Batch_id=937 Accuracy=88.50: 100%|██████████| 938/938 [00:21<00:00, 44.17it/s]



Test set: Average loss: 0.0604, Accuracy: 9873/10000 (98.73%)



## Model 2

In [11]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_2().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 10, 24, 24]             160
         MaxPool2d-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 10, 10, 10]             900
             ReLU-10           [-1, 10, 10, 10]               0
      BatchNorm2d-11           [-1, 10, 10, 10]              20
           Conv2d-12             [-1, 16, 8, 8]           1,440
             ReLU-13             [-1, 16, 8, 8]               0
      BatchNorm2d-14             [-1, 1

In [12]:
model = Model_2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

for epoch in range(0, 15):
    print(f"EPOCH: {epoch+1}")
    utils.train(model, device, train_loader, optimizer, epoch)
    utils.test(model, device, test_loader)

EPOCH: 1


Loss=0.046187739819288254 Batch_id=937 Accuracy=93.98: 100%|██████████| 938/938 [00:21<00:00, 43.76it/s]



Test set: Average loss: 0.0617, Accuracy: 9809/10000 (98.09%)

EPOCH: 2


Loss=0.021620821207761765 Batch_id=937 Accuracy=97.91: 100%|██████████| 938/938 [00:21<00:00, 44.48it/s]



Test set: Average loss: 0.0451, Accuracy: 9862/10000 (98.62%)

EPOCH: 3


...


Test set: Average loss: 0.0283, Accuracy: 9913/10000 (99.13%)

EPOCH: 14


Loss=0.0032173716463148594 Batch_id=937 Accuracy=99.39: 100%|██████████| 938/938 [00:22<00:00, 41.93it/s]



Test set: Average loss: 0.0310, Accuracy: 9908/10000 (99.08%)

EPOCH: 15


Loss=0.0001716601982479915 Batch_id=937 Accuracy=99.47: 100%|██████████| 938/938 [00:21<00:00, 42.97it/s]



Test set: Average loss: 0.0311, Accuracy: 9910/10000 (99.10%)



## Model 3

In [13]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Model_3().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,152
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 10, 24, 24]             160
         MaxPool2d-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 10, 10, 10]             900
             ReLU-10           [-1, 10, 10, 10]               0
      BatchNorm2d-11           [-1, 10, 10, 10]              20
           Conv2d-12             [-1, 16, 8, 8]           1,440
             ReLU-13             [-1, 16, 8, 8]               0
      BatchNorm2d-14             [-1, 1

In [14]:
model = Model_3().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')

for epoch in range(0, 15):
    print(f"EPOCH: {epoch+1}")
    utils.train(model, device, train_loader, optimizer, epoch)
    utils.test(model, device, test_loader)

EPOCH: 1


Loss=0.06166558712720871 Batch_id=937 Accuracy=94.13: 100%|██████████| 938/938 [00:21<00:00, 42.71it/s]



Test set: Average loss: 0.0531, Accuracy: 9837/10000 (98.37%)

EPOCH: 2


Loss=0.1172497346997261 Batch_id=937 Accuracy=98.07: 100%|██████████| 938/938 [00:21<00:00, 43.41it/s]



Test set: Average loss: 0.0628, Accuracy: 9819/10000 (98.19%)

EPOCH: 3


...


Test set: Average loss: 0.0313, Accuracy: 9916/10000 (99.16%)

EPOCH: 14


Loss=0.003181066131219268 Batch_id=937 Accuracy=99.42: 100%|██████████| 938/938 [00:20<00:00, 45.59it/s]



Test set: Average loss: 0.0296, Accuracy: 9916/10000 (99.16%)

EPOCH: 15


Loss=0.0017103513237088919 Batch_id=937 Accuracy=99.44: 100%|██████████| 938/938 [00:20<00:00, 45.31it/s]



Test set: Average loss: 0.0298, Accuracy: 9920/10000 (99.20%)

